In [1]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    import pandas as pd
    import pandas_datareader.data as web
    import pickle 
    import requests
    import os
    from datetime import date

In [2]:
name = 'Moving Average agent'
image_path = 'D:/kenneth/agent/images/'
path = 'D:/kenneth/trading/'
run_date=open(path+'run_date.txt').read()
start=('2019-01-01')
end=('2019-12-31')
days=30

In [3]:
if not os.path.exists(image_path+name):
        os.makedirs(image_path+name) 
with open('D:/kenneth/trading/sp500/sp500tickers.txt','r') as f:
    tickers=[line.rstrip('\n') for line in f]

In [4]:
def buy_stock(
    real_movement,
    signal,
    initial_money = 10000,
    max_buy = 1,
    max_sell = 1,
):
    """
    real_movement = actual movement in the real world
    delay = how much interval you want to delay to change our decision from buy to sell, vice versa
    initial_state = 1 is buy, 0 is sell
    initial_money = 1000, ignore what kind of currency
    max_buy = max quantity for share to buy
    max_sell = max quantity for share to sell
    """
    starting_money = initial_money
    states_sell = []
    states_buy = []
    current_inventory = 0

    def buy(i, initial_money, current_inventory):
        shares = initial_money // real_movement[i]
        if shares < 1:
            pass
#             print(
#                 'day %d: total balances %f, not enough money to buy a unit price %f'
#                 % (i, initial_money, real_movement[i])
#             )
        else:
            if shares > max_buy:
                buy_units = max_buy
            else:
                buy_units = shares
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
#             print(
#                 'day %d: buy %d units at price %f, total balance %f'
#                 % (i, buy_units, buy_units * real_movement[i], initial_money)
#             )
            states_buy.append(0)
        return initial_money, current_inventory

    for i in range(real_movement.shape[0] - int(0.025 * len(df))):
        state = signal[i]
        if state == 1:
            initial_money, current_inventory = buy(
                i, initial_money, current_inventory
            )
            states_buy.append(i)
        elif state == -1:
            if current_inventory == 0:
                pass
#                     print('day %d: cannot sell anything, inventory 0' % (i))
            else:
                if current_inventory > max_sell:
                    sell_units = max_sell
                else:
                    sell_units = current_inventory
                current_inventory -= sell_units
                total_sell = sell_units * real_movement[i]
                initial_money += total_sell
                try:
                    invest = (
                        (real_movement[i] - real_movement[states_buy[-1]])
                        / real_movement[states_buy[-1]]
                    ) * 100
                except:
                    invest = 0
#                 print(
#                     'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
#                     % (i, sell_units, total_sell, invest, initial_money)
#                 )
            states_sell.append(i)
    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money
    return states_buy, states_sell, total_gains, invest

In [5]:
def plot_all(ticker):
    close = df['Close']
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=10, color='g', label = 'buying signal', markevery = states_buy)
    plt.plot(close, 'v', markersize=10, color='r', label = 'selling signal', markevery = states_sell)
    plt.title(name+'\n'+'Ticker: '+ticker)
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'.png', facecolor='yellowgreen', dpi=100)
    #plt.show()
    plt.close()

In [6]:
def display_only(days):
    new_buy = []
    new_sell = []
    display_days = days
    total_days= df.shape[0]
    display_from = total_days-display_days
    for i in(states_buy):
        if i > display_from:
            adj = i- display_from
            new_buy.append(adj)
    for i in(states_sell):
        if i > display_from:
            adj = i- display_from
            new_sell.append(adj)
    
    return(new_buy, new_sell)

In [7]:
def plot_by_days(days, ticker):
    close = df['Close'][-days:]
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=12, color='g',alpha=1.0,label = 'buying signal', markevery = new_buy)
    plt.plot(close, 'v', markersize=12, color='r',alpha=1.0, label = 'selling signal', markevery = new_sell)
    plt.title(name+'\n'+'Last '+str(days)+' Daily Trade Recommendations')
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'_'+str(days)+'.png', facecolor='yellowgreen', dpi=100)
    #plt.show()
    plt.close()

In [8]:
def save_signals():
    path2 = image_path+name+'/signal/'
    if not os.path.exists(path2):
        os.makedirs(path2)
        
    if  (len(states_buy)!= 0) and (len(states_sell) != 0):  
        if int(states_buy[-1])>int(states_sell[-1]):
            last_signal = 'Trading Buy'
            signal_day = str(df.index[states_buy[-1]]).split(' ')[0]
        else:
            last_signal = 'Trading Sell'
            signal_day = str(df.index[states_sell[-1]]).split(' ')[0]
        
    elif len(states_buy)== 0 and len(states_sell)>0:
        last_signal = 'Trading Sell'
        signal_day = str(df.index[states_sell[-1]]).split(' ')[0]
    elif len(states_sell)== 0 and len(states_buy)>0:
        last_signal = 'Trading Buy'
        signal_day = str(df.index[states_buy[-1]]).split(' ')[0]
    elif len(states_sell)== 0 and len(states_buy)==0:
        last_signal = 'None'
        signal_day = 'None'  
        
    with open(path2+ticker+'.txt','w') as f:
        print(
            'model_run,','signal_day,','last_signal,','total_gains,','invest,','data_start','\n',
              str(date.today()),',',signal_day,',',last_signal,',',total_gains,',',invest,',',str(start),
              file=f)

In [9]:
for ticker in tickers:
    df = pd.read_csv(path+'sp500/'+run_date+'/'+ticker+'.csv', index_col=0, parse_dates=True)
    df =df[start:end]
    print('Started ticker:', ticker)
    count = int(np.ceil(len(df) * 0.1))
    short_window = int(0.025 * len(df))
    long_window = int(0.05 * len(df))

    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0.0

    signals['short_ma'] = df['Close'].rolling(window=short_window, min_periods=1, center=False).mean()
    signals['long_ma'] = df['Close'].rolling(window=long_window, min_periods=1, center=False).mean()

    signals['signal'][short_window:] = np.where(signals['short_ma'][short_window:] 
                                                > signals['long_ma'][short_window:], 1.0, 0.0)   
    signals['positions'] = signals['signal'].diff()

    states_buy, states_sell, total_gains, invest = buy_stock(df.Close, signals['positions'])
    save_signals()  
    plot_all(ticker)
#     new_buy, new_sell = display_only(days)
#     plot_by_days(days, ticker)


Started ticker: A
Started ticker: AAL
Started ticker: AAP
Started ticker: AAPL
Started ticker: ABBV
Started ticker: ABC
Started ticker: ABMD
Started ticker: ABT
Started ticker: ACN
Started ticker: ADBE
Started ticker: ADI
Started ticker: ADM
Started ticker: ADP
Started ticker: ADS
Started ticker: ADSK
Started ticker: AEE
Started ticker: AEP
Started ticker: AES
Started ticker: AFL
Started ticker: AGN
Started ticker: AIG
Started ticker: AIV
Started ticker: AIZ
Started ticker: AJG
Started ticker: AKAM
Started ticker: ALB
Started ticker: ALGN
Started ticker: ALK
Started ticker: ALL
Started ticker: ALLE
Started ticker: ALXN
Started ticker: AMAT
Started ticker: AMCR
Started ticker: AMD
Started ticker: AME
Started ticker: AMG
Started ticker: AMGN
Started ticker: AMP
Started ticker: AMT
Started ticker: AMZN
Started ticker: ANET
Started ticker: ANSS
Started ticker: ANTM
Started ticker: AON
Started ticker: AOS
Started ticker: APA
Started ticker: APD
Started ticker: APH
Started ticker: APTV
Start

Started ticker: SBAC
Started ticker: SBUX
Started ticker: SCHW
Started ticker: SEE
Started ticker: SHW
Started ticker: SIVB
Started ticker: SJM
Started ticker: SLB
Started ticker: SLG
Started ticker: SNA
Started ticker: SNPS
Started ticker: SO
Started ticker: SPG
Started ticker: SPGI
Started ticker: SPY
Started ticker: SRE
Started ticker: STI
Started ticker: STT
Started ticker: STX
Started ticker: STZ
Started ticker: SWK
Started ticker: SWKS
Started ticker: SYF
Started ticker: SYK
Started ticker: SYMC
Started ticker: SYY
Started ticker: T
Started ticker: TAP
Started ticker: TDG
Started ticker: TEL
Started ticker: TFX
Started ticker: TGT
Started ticker: TIF
Started ticker: TJX
Started ticker: TMO
Started ticker: TMUS
Started ticker: TPR
Started ticker: TRIP
Started ticker: TROW
Started ticker: TRV
Started ticker: TSCO
Started ticker: TSN
Started ticker: TSS
Started ticker: TTWO
Started ticker: TWTR
Started ticker: TXN
Started ticker: TXT
Started ticker: UA
Started ticker: UAA
Started ti